In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
road=cv2.imread('../DATA/road_image.jpg')

In [6]:
road_copy=np.copy(road)

In [7]:
road.shape

(600, 800, 3)

In [8]:
marker_image=np.zeros(road.shape[:2],dtype=np.int32)

In [9]:
segments=np.zeros(road.shape,dtype=np.uint8)

In [10]:
marker_image.shape

(600, 800)

In [11]:
segments.shape

(600, 800, 3)

In [12]:
from matplotlib import cm

In [13]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [14]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(0)[:3])*255)

In [15]:
colors=[]
for i in range(10):
    colors.append(create_rgb(i))

In [16]:
colors

[(31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0),
 (31.0, 119.0, 180.0)]

In [17]:
# Global variables

In [18]:
# no of markers
n_markers=10
# color choice
current_marker=1
# markers updated by watershed
marks_updated=False

In [19]:
def mouse_callback(event, x, y, flags, param):
    global marks_updated 

    if event == cv2.EVENT_LBUTTONDOWN:
        
        # TRACKING FOR MARKERS
        cv2.circle(marker_image, (x, y), 10, (current_marker), -1)
        
        # DISPLAY ON USER IMAGE
        cv2.circle(road_copy, (x, y), 10, colors[current_marker], -1)
        marks_updated = True

In [20]:
cv2.namedWindow('Road Image')
cv2.setMouseCallback('Road Image', mouse_callback)

while True:
    
    # SHow the 2 windows
    cv2.imshow('WaterShed Segments', segments)
    cv2.imshow('Road Image', road_copy)
        
        
    # Close everything if Esc is pressed
    k = cv2.waitKey(1)

    if k == 27:
        break
        
    # Clear all colors and start over if 'c' is pressed
    elif k == ord('c'):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[0:2], dtype=np.int32)
        segments = np.zeros(road.shape,dtype=np.uint8)
        
    # If a number 0-9 is chosen index the color
    elif k > 0 and chr(k).isdigit():
        # chr converts to printable digit
        
        current_marker  = int(chr(k))
        
    # If we clicked somewhere, call the watershed algorithm on our chosen markers
    if marks_updated:
        
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        
        segments = np.zeros(road.shape,dtype=np.uint8)
        
        for color_ind in range(n_markers):
            segments[marker_image_copy == (color_ind)] = colors[color_ind]
        
        marks_updated = False
        
cv2.destroyAllWindows()